In [33]:
name="AuAuGradsobolgroup"

In [34]:
import os
import numpy as np
import seaborn as sns
import pandas as pd
import math
import matplotlib.pyplot as plt


from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessRegressor as gpr
from sklearn.gaussian_process import kernels as krnl
import scipy.stats as st
from scipy import optimize

from multiprocessing import Pool
from multiprocessing import cpu_count

import time

### Setup working folders


In [35]:
# Where to save the figures and data files
PROJECT_ROOT_DIR = "Results"
FIGURE_ID = "Results/FigureFiles"
DATA_ID = "DataFiles/"

In [36]:
if not os.path.exists(PROJECT_ROOT_DIR):
    os.mkdir(PROJECT_ROOT_DIR)

if not os.path.exists(FIGURE_ID):
    os.makedirs(FIGURE_ID)

if not os.path.exists(DATA_ID):
    os.makedirs(DATA_ID)

def image_path(fig_id):
    return os.path.join(FIGURE_ID, fig_id)

def data_path(dat_id):
    return os.path.join(DATA_ID, dat_id)

def save_fig(fig_id):
    plt.savefig(image_path(fig_id) + ".png", format='png')

In [37]:
# Bounds for parametrs in the emulator are same as prior ranges so
prior_df = pd.read_csv(filepath_or_buffer="DataFiles/PbPb2760_prior", index_col=0)

In [38]:
# Design points
design = pd.read_csv(filepath_or_buffer="DataFiles/PbPb2760_design")
design_Au = pd.read_csv(filepath_or_buffer="DataFiles/AuAu200_design")
design_validation = pd.read_csv(filepath_or_buffer="DataFiles/AuAu200_validation_design")

In [39]:
#Simulation outputs at the design points
simulation_df = []
simulation_df_Au = []
for idf in range(0,4):
    simulation_df_Au.append(pd.read_csv(filepath_or_buffer=f"DataFiles/AuAu200_simulation_{idf}"))
    simulation_df.append(pd.read_csv(filepath_or_buffer=f"DataFiles/PbPb2760_simulation_{idf}"))
    simulation_df[idf]=simulation_df[idf][simulation_df_Au[0].keys()]

    #simulation_sd_df.append(pd.read_csv(filepath_or_buffer=f"DataFiles/PbPb2760_simulation_error_{idf}"))

In [40]:
simulation_df_Au_validation=pd.read_csv(filepath_or_buffer=f"DataFiles/AuAU200_validation_simulation")


In [41]:
simulation_df_Au_validation.head()

,dN_dy_pion[0 5],dN_dy_pion[ 5 10],dN_dy_pion[10 20],dN_dy_pion[20 30],dN_dy_pion[30 40],dN_dy_pion[40 50],dN_dy_kaon[0 5],dN_dy_kaon[ 5 10],dN_dy_kaon[10 20],dN_dy_kaon[20 30],...,v22[10 20],v22[20 30],v22[30 40],v22[40 50],v32[0 5],v32[ 5 10],v32[10 20],v32[20 30],v32[30 40],v32[40 50]
0,679.470390,528.721444,377.220283,247.710011,154.809635,89.625091,117.671856,93.069604,66.588343,44.366692,...,0.069750,0.086235,0.092955,0.102498,0.018113,0.023048,0.024176,0.023233,0.024741,0.021809
1,506.610998,389.387330,268.567238,171.141353,101.217369,53.278526,88.686814,68.846050,48.132513,31.177262,...,0.056837,0.074908,0.077329,0.081317,0.012109,0.012531,0.015516,0.013537,0.014320,0.011219
2,601.920037,497.966578,385.784684,266.620161,183.978890,122.591259,106.024508,88.246614,69.338185,48.241979,...,0.048425,0.061207,0.072616,0.072245,0.014845,0.016672,0.020201,0.022260,0.023141,0.024597
3,514.680557,388.143313,281.274163,163.008529,98.517387,51.071891,90.109347,68.732488,50.499513,29.909462,...,0.068922,0.095341,0.106568,0.106427,0.019101,0.024966,0.024701,0.026831,0.021940,0.019515
4,643.292840,546.781515,417.832459,290.655091,194.170271,119.372862,112.565223,96.450635,74.064022,51.989790,...,0.053201,0.069107,0.084691,0.086855,0.016178,0.014876,0.017368,0.021080,0.020938,0.020297


In [42]:
X = design.values
XAu = design_Au.values
Xv = design_validation.values

In [43]:
X

array([[15.81754, -0.06769,  0.98153, ...,  0.1527 ,  4.79271,  0.14759],
       [13.4397 ,  0.12337,  1.34278, ...,  0.29229,  5.81536,  0.14498],
       [15.84174,  0.13791,  1.06155, ...,  0.07515,  3.39156,  0.15274],
       ...,
       [18.48707, -0.63453,  1.56869, ..., -0.48639,  2.53521,  0.1541 ],
       [12.63315, -0.35414,  1.87613, ...,  0.30156,  6.48054,  0.13593],
       [11.07209,  0.45782,  0.32487, ..., -0.1866 ,  3.98769,  0.15467]])

In [44]:
XAu

array([[ 6.90877, -0.06769,  0.98153, ...,  0.1527 ,  4.79271,  0.14759],
       [ 5.71985,  0.12337,  1.34278, ...,  0.29229,  5.81536,  0.14498],
       [ 6.92087,  0.13791,  1.06155, ...,  0.07515,  3.39156,  0.15274],
       ...,
       [ 8.24353, -0.63453,  1.56869, ..., -0.48639,  2.53521,  0.1541 ],
       [ 5.31658, -0.35414,  1.87613, ...,  0.30156,  6.48054,  0.13593],
       [ 4.53605,  0.45782,  0.32487, ..., -0.1866 ,  3.98769,  0.15467]])

In [45]:
df_clms=simulation_df[1].keys()

### Normalize all the other models using mean and variance of observable in the PbPb 2.76 TeV Grad model

In [46]:
#normalize data with respect to lower fidelity
s_l = StandardScaler()
x = simulation_df[0].values
s_l.fit(x)
for idf in range(0,4):
    x_tmp = simulation_df[idf].values
    simulation_df[idf]= pd.DataFrame(s_l.transform(x_tmp),columns=df_clms)
    
    x_tmp = simulation_df_Au[idf].values
    simulation_df_Au[idf]= pd.DataFrame(s_l.transform(x_tmp),columns=df_clms)
    

#diff = np.array(prior_df.loc['max'].values - prior_df.loc['min'].values ).reshape(1,-1)
#diff_mat = np.repeat(diff,X.shape[0],axis=0)
#print(f'Shape of diff matt {diff_mat.shape}')
#X= np.divide(X,diff_mat)

In [47]:
print('normalizing validation data')
x_tmp = simulation_df_Au_validation.values
print(x_tmp)
simulation_df_Au_validation=pd.DataFrame(s_l.transform(x_tmp),columns=df_clms)

normalizing validation data
[[6.79470390e+02 5.28721444e+02 3.77220283e+02 ... 2.32333907e-02
  2.47413895e-02 2.18087984e-02]
 [5.06610998e+02 3.89387330e+02 2.68567238e+02 ... 1.35370801e-02
  1.43197658e-02 1.12191806e-02]
 [6.01920037e+02 4.97966578e+02 3.85784684e+02 ... 2.22597765e-02
  2.31408449e-02 2.45965445e-02]
 ...
 [3.80396107e+02 2.87092933e+02 2.03508279e+02 ... 2.24272344e-02
  2.13253502e-02 1.45836949e-02]
 [7.83618121e+02 6.69338022e+02 5.19107230e+02 ... 1.65346961e-02
  1.72146242e-02 1.63043144e-02]
 [5.44399579e+02 4.26784811e+02 3.16365101e+02 ... 2.75511984e-02
  2.93476172e-02 2.38313993e-02]]


In [48]:
simulation_df_Au_validation.head()

,dN_dy_pion[0 5],dN_dy_pion[ 5 10],dN_dy_pion[10 20],dN_dy_pion[20 30],dN_dy_pion[30 40],dN_dy_pion[40 50],dN_dy_kaon[0 5],dN_dy_kaon[ 5 10],dN_dy_kaon[10 20],dN_dy_kaon[20 30],...,v22[10 20],v22[20 30],v22[30 40],v22[40 50],v32[0 5],v32[ 5 10],v32[10 20],v32[20 30],v32[30 40],v32[40 50]
0,-2.523938,-2.507397,-2.472066,-2.315738,-2.195433,-2.086820,-2.551722,-2.518510,-2.501256,-2.337592,...,0.423260,0.236187,0.022224,0.153342,-0.289278,0.046856,-0.152791,-0.512065,-0.435370,-0.663728
1,-3.196920,-3.136012,-3.078607,-2.887698,-2.749855,-2.632324,-3.225675,-3.169963,-3.111651,-2.916424,...,-0.332449,-0.272415,-0.584451,-0.597102,-1.296596,-1.528788,-1.336980,-1.631970,-1.491588,-1.636714
2,-2.825860,-2.646150,-2.424257,-2.174481,-1.893672,-1.592051,-2.822543,-2.648216,-2.410310,-2.167521,...,-0.824739,-0.887580,-0.767446,-0.918534,-0.837533,-0.908399,-0.696357,-0.624515,-0.597583,-0.407587
3,-3.165504,-3.141624,-3.007672,-2.948449,-2.777787,-2.665442,-3.192599,-3.173017,-3.033367,-2.972063,...,0.374814,0.645073,0.550757,0.292545,-0.123634,0.334304,-0.080955,-0.096546,-0.719297,-0.874527
4,-2.664786,-2.425918,-2.245354,-1.994942,-1.788240,-1.640354,-2.670460,-2.427583,-2.254011,-2.003044,...,-0.545203,-0.532886,-0.298640,-0.400885,-0.613893,-1.177382,-1.083634,-0.760832,-0.820880,-0.802603


In [49]:
simulation_df[0].head()

,dN_dy_pion[0 5],dN_dy_pion[ 5 10],dN_dy_pion[10 20],dN_dy_pion[20 30],dN_dy_pion[30 40],dN_dy_pion[40 50],dN_dy_kaon[0 5],dN_dy_kaon[ 5 10],dN_dy_kaon[10 20],dN_dy_kaon[20 30],...,v22[10 20],v22[20 30],v22[30 40],v22[40 50],v32[0 5],v32[ 5 10],v32[10 20],v32[20 30],v32[30 40],v32[40 50]
0,0.619257,0.530218,0.327470,0.272370,0.033842,-0.056312,0.614684,0.521940,0.333326,0.299804,...,0.615527,0.517000,0.657858,0.602001,-0.150874,0.321698,0.264980,0.319903,0.324693,0.463557
1,-0.180698,-0.092690,0.083062,0.245721,0.050315,0.133230,-0.200765,-0.094240,0.068796,0.233722,...,-0.621244,-0.382229,-0.340909,-0.443330,-0.150021,-0.367147,-0.030290,-0.272368,-0.140120,-0.342011
2,1.037999,1.022160,0.995952,0.931593,0.987230,0.988813,0.986484,1.001289,0.975533,0.906810,...,-0.768037,-0.625974,-0.694006,-0.733197,-0.561079,-0.897921,-0.666548,-0.737213,-0.796565,-0.605217
3,0.137978,0.101764,0.162907,0.297756,0.415575,0.474692,0.160239,0.119394,0.162974,0.302322,...,-0.487681,-0.416800,-0.429595,-0.465382,-0.782748,-0.794482,-0.852256,-0.773484,-0.507853,-0.498973
4,0.692222,0.839457,0.934473,0.870276,1.038612,1.065842,0.639250,0.758631,0.873731,0.801596,...,-0.545879,-0.513280,-0.489739,-0.425560,0.370203,0.074837,-0.009571,0.044080,-0.003329,0.008152


In [50]:
simulation_df_Au[0].head()

,dN_dy_pion[0 5],dN_dy_pion[ 5 10],dN_dy_pion[10 20],dN_dy_pion[20 30],dN_dy_pion[30 40],dN_dy_pion[40 50],dN_dy_kaon[0 5],dN_dy_kaon[ 5 10],dN_dy_kaon[10 20],dN_dy_kaon[20 30],...,v22[10 20],v22[20 30],v22[30 40],v22[40 50],v32[0 5],v32[ 5 10],v32[10 20],v32[20 30],v32[30 40],v32[40 50]
0,-2.466885,-2.340405,-2.210642,-2.104270,-1.973155,-1.861972,-2.490622,-2.376828,-2.227500,-2.117685,...,-0.315905,-0.197261,-0.022988,-0.190521,-0.671299,-0.749780,-0.650099,-0.583695,-0.545412,-0.612321
1,-2.933848,-2.754448,-2.628082,-2.468001,-2.315428,-2.197366,-2.967860,-2.790162,-2.680301,-2.518459,...,-0.816157,-0.921607,-0.915112,-1.089463,-0.962587,-0.990708,-1.223701,-1.014291,-1.188219,-1.217243
2,-2.325311,-2.236444,-2.154113,-1.995829,-1.872947,-1.757504,-2.370010,-2.284442,-2.189029,-2.030663,...,-1.072326,-1.129969,-1.213798,-1.305465,-1.393720,-1.364287,-1.488997,-1.364108,-1.397129,-1.374673
3,-2.667782,-2.576655,-2.447551,-2.262546,-2.102623,-1.978883,-2.699629,-2.607532,-2.490498,-2.296631,...,-0.954619,-1.035027,-0.906722,-1.007988,-1.443761,-1.425859,-1.578666,-1.373466,-1.307209,-1.347564
4,-2.331930,-2.154519,-1.957723,-1.757908,-1.531543,-1.370540,-2.380620,-2.220087,-2.015272,-1.805281,...,-1.028587,-0.882366,-1.016598,-0.971075,-0.726944,-0.616042,-0.817584,-0.878947,-0.625988,-0.632455


In [51]:
for idf in range(0,4):
    Y = simulation_df[idf].values
    print('###########################\n for PbPb 2760')
    print( "X.shape : "+ str(X.shape) )
    print( "Y.shape : "+ str(Y.shape) )

    print('###########################\n for AuAU 200')
    Y_Au = simulation_df_Au[idf].values
    print( "X.shape : "+ str(XAu.shape) )
    print( "Y.shape : "+ str(Y.shape) )

###########################
 for PbPb 2760
X.shape : (473, 17)
Y.shape : (473, 36)
###########################
 for AuAU 200
X.shape : (473, 17)
Y.shape : (473, 36)
###########################
 for PbPb 2760
X.shape : (473, 17)
Y.shape : (473, 36)
###########################
 for AuAU 200
X.shape : (473, 17)
Y.shape : (473, 36)
###########################
 for PbPb 2760
X.shape : (473, 17)
Y.shape : (473, 36)
###########################
 for AuAU 200
X.shape : (473, 17)
Y.shape : (473, 36)
###########################
 for PbPb 2760
X.shape : (473, 17)
Y.shape : (473, 36)
###########################
 for AuAU 200
X.shape : (473, 17)
Y.shape : (473, 36)


In [52]:
#Model parameter names in Latex compatble form
model_param_dsgn = ['$N$[$200$GeV]',
 '$p$',
 '$\\sigma_k$',
 '$w$ [fm]',
 '$d_{\\mathrm{min}}$ [fm]',
 '$\\tau_R$ [fm/$c$]',
 '$\\alpha$',
 '$T_{\\eta,\\mathrm{kink}}$ [GeV]',
 '$a_{\\eta,\\mathrm{low}}$ [GeV${}^{-1}$]',
 '$a_{\\eta,\\mathrm{high}}$ [GeV${}^{-1}$]',
 '$(\\eta/s)_{\\mathrm{kink}}$',
 '$(\\zeta/s)_{\\max}$',
 '$T_{\\zeta,c}$ [GeV]',
 '$w_{\\zeta}$ [GeV]',
 '$\\lambda_{\\zeta}$',
 '$b_{\\pi}$',
 '$T_{\\mathrm{sw}}$ [GeV]']

In [53]:
observables = ['dNch_deta',
 'dET_deta',
 'dN_dy_pion',
 'dN_dy_kaon',
 'dN_dy_proton',
 'mean_pT_pion',
 'mean_pT_kaon',
 'mean_pT_proton',
 'pT_fluct',
 'v22',
 'v32',
 'v42']

In [54]:
observables_latex_2 = ['$\\frac{dN_{ch}}{d\\eta}$',
 '$\\frac{dE_T}{d\\eta}$',
 '$\\frac{dN_{\\pi}}{dy}$',
 '$\\frac{dN_{K}}{dy}$',
 '$\\frac{dN_{P}}{dy}$',
 '$\\langle pT_{\\pi} \\rangle$',
 '$\\langle pT_{K} \\rangle$',
 '$\\langle pT_{P} \\rangle$',
 '$\\frac{\\delta p_T}{\\langle p_T \\rangle}$',
 '$v_2${2}',
 '$v_3${2}',
 '$v_4${2}']

In [55]:
observables_latex = ['$dN_{ch} / d\\eta$',
 '$dE_T / d\\eta$',
 '${dN_{\\pi}} / {dy}$',
 '${dN_{K}} / {dy}$',
 '${dN_{P}} / {dy}$',
 '$\\langle p_{T, \\pi} \\rangle$',
 '$\\langle p_{T, K} \\rangle$',
 '$\\langle p_{T, P} \\rangle$',
 '${\\delta p_T} / {\\langle p_T \\rangle}$',
 '$v_2${2}',
 '$v_3${2}',
 '$v_4${2}']

### Observables considered in this analysis

In [56]:
simulation_df[0].keys()

Index(['dN_dy_pion[0 5]', 'dN_dy_pion[ 5 10]', 'dN_dy_pion[10 20]',
       'dN_dy_pion[20 30]', 'dN_dy_pion[30 40]', 'dN_dy_pion[40 50]',
       'dN_dy_kaon[0 5]', 'dN_dy_kaon[ 5 10]', 'dN_dy_kaon[10 20]',
       'dN_dy_kaon[20 30]', 'dN_dy_kaon[30 40]', 'dN_dy_kaon[40 50]',
       'mean_pT_pion[0 5]', 'mean_pT_pion[ 5 10]', 'mean_pT_pion[10 20]',
       'mean_pT_pion[20 30]', 'mean_pT_pion[30 40]', 'mean_pT_pion[40 50]',
       'mean_pT_kaon[0 5]', 'mean_pT_kaon[ 5 10]', 'mean_pT_kaon[10 20]',
       'mean_pT_kaon[20 30]', 'mean_pT_kaon[30 40]', 'mean_pT_kaon[40 50]',
       'v22[0 5]', 'v22[ 5 10]', 'v22[10 20]', 'v22[20 30]', 'v22[30 40]',
       'v22[40 50]', 'v32[0 5]', 'v32[ 5 10]', 'v32[10 20]', 'v32[20 30]',
       'v32[30 40]', 'v32[40 50]'],
      dtype='object')

In [57]:
observables_choosen = ['dN_dy_pion[0 5]',
 'dN_dy_pion[40 50]',
# 'dN_dy_kaon[0 5]',
# 'dN_dy_kaon[60 70]',
# 'dN_dy_proton[0 5]',
# 'dN_dy_proton[60 70]',
 'mean_pT_pion[0 5]',
 'mean_pT_pion[40 50]',
# 'mean_pT_kaon[0 5]',
# 'mean_pT_kaon[60 70]',               
# 'mean_pT_proton[0 5]',
# 'mean_pT_proton[60 70]',
 'v22[0 5]',
  'v22[40 50]']

###  Build the source emulators and find length scales and white noise that we can use in multifidelity code to fix hyper parameters.

In [58]:
import GPy
import emukit.multi_fidelity
from emukit.model_wrappers.gpy_model_wrappers import GPyMultiOutputWrapper
from emukit.multi_fidelity.models import GPyLinearMultiFidelityModel
## Convert lists of arrays to ndarrays augmented with fidelity indicators
from sklearn.model_selection import train_test_split
from emukit.multi_fidelity.convert_lists_to_array import convert_x_list_to_array, convert_xy_lists_to_arrays
from sklearn.model_selection import KFold
#from emukit.multi_fidelity.models.non_linear_multi_fidelity_model import make_non_linear_kernels, NonLinearMultiFidelityModel
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib

In [59]:
#low_gp_model = GPy.models.GPRegression.optimize_restarts()

    

In [60]:
# GP_dic={}
# n_opt = 10
# n_proc=10
# for selected_observable in observables_choosen:
#     Y_l=simulation_df[0][selected_observable].values.reshape(-1,1)
#     Y_h=simulation_df_Au[0][selected_observable].values.reshape(-1,1)
#     Y_v=simulation_df_Au_validation[selected_observable].values.reshape(-1,1)
    
#     x_train_h, x_test_h, y_train_h, y_test_h = XAu, Xv, Y_h, Y_v
#     x_train_l, y_train_l = X, Y_l
#     print(selected_observable)
#     kernel = GPy.kern.RBF(input_dim=17, ARD=True)
#     low_gp_model = GPy.models.GPRegression(x_train_l,y_train_l, kernel)
#         #high_gp_model.Gaussian_noise.fix(0)

#         ## Fit the GP model
    
#     low_gp_model.optimize_restarts(n_opt, verbose=True, parallel=True, num_processes=n_proc)
#     print(low_gp_model)
#     GP_dic[selected_observable]=low_gp_model

In [61]:
#import pickle
#with open(data_path('PbPbGradforAusobol'),"wb") as f:
#    pickle.dump(GP_dic,f)

In [62]:
import pickle
GP_dic = pickle.load(open(data_path('PbPbGradforAusobol'),"rb"))

### Transfer Learning using Emukit

In [63]:
def run_train_and_validation(x_train_s, x_train_t, x_test_t, y_train_s, y_train_t, y_test_t, obs_name):
    """ Train two types of emulators. And perform validation on a given set
            1. Linear multifidelity GPs
            2. Standard GP
        =============================
        Return:
            [3 x 2 ndarray,rho value], [residue] of Transfer Learning model
            First row has MSE
            Second row has R2 scores
            Third row has the White noise variance
        """
    print('########################')
    print(obs_name)
    X_train, Y_train = convert_xy_lists_to_arrays([x_train_s, x_train_t], [y_train_s, y_train_t])
    n_opt = 10
    n_proc = 10
    
    ## Construct a linear multi-fidelity model

    #kernels = [GPy.kern.RBF(17, ARD=True), GPy.kern.RBF(17, ARD=True)]
    kernels = [GPy.kern.RBF(17, ARD=True), GPy.kern.RBF(17, ARD=True)]
    lin_tl_kernel = emukit.multi_fidelity.kernels.LinearMultiFidelityKernel(kernels)
    gpy_lin_tl_model = GPyLinearMultiFidelityModel(X_train, Y_train, lin_tl_kernel, n_fidelities=2)
    
    #Set the parameters in the Source Gaussian Process
    
    rbf_l_scales = GP_dic[obs_name].rbf.lengthscale.values
    rbf_var = GP_dic[obs_name].rbf.variance.values[0]
    wn = GP_dic[obs_name].Gaussian_noise.variance.values[0]
    
    gpy_lin_tl_model.multifidelity.rbf.lengthscale.fix(value=rbf_l_scales)
    gpy_lin_tl_model.multifidelity.rbf.variance.fix(value=rbf_var)
    gpy_lin_tl_model.mixed_noise.Gaussian_noise.fix(wn)
    #gpy_lin_mf_model.mixed_noise.Gaussian_noise_1.fix(0)


    ## Wrap the model using the given 'GPyMultiOutputWrapper'
    lin_tl_model = GPyMultiOutputWrapper(gpy_lin_tl_model, 2, n_optimization_restarts=n_opt,
                                         parallel=True, num_processes=n_proc)

    ## Fit the model
    st_l = time.time()
    lin_tl_model.optimize()
    et_l = time.time()
    
    #print("parameter array provided")
    #print(f'{rbf_l_scales} noise {wn}')
    
    if np.allclose(lin_tl_model.gpy_model.multifidelity.rbf.lengthscale.values, rbf_l_scales):
          print('RBF lengthscales are the same for low fidelity after optimization')
    if wn == lin_tl_model.gpy_model.mixed_noise.Gaussian_noise.variance.values[0]:
          print('White noise are the same after optimization')
    print('parameter array of multifidelity linear')
    print(lin_tl_model.gpy_model.param_array)
    print(f'time for linear mf optimization {et_l-st_l}')
    ## Create standard GP model using only high-fidelity data

    #kernel = GPy.kern.RBF(input_dim=17, ARD=True)
    #sd_gp_model = GPy.models.GPRegression(x_train_t,y_train_t, kernel)
    #high_gp_model.Gaussian_noise.fix(0)

    ## Fit the GP model
    
    #sd_gp_model.optimize_restarts(n_opt, verbose=True, parallel=True, num_processes=n_proc)
    
    x_temp = convert_x_list_to_array([x_test_t,x_test_t])
    x_test_t_idf_index = x_temp[x_test_t.shape[0]:,:]
    x_test_s_idf_index = x_temp[0:x_test_t.shape[0],:]
    
    
    #Transfer learning predictions and validation score calculations
    t_mean_lin_tl_model, t_var_lin_tl_model = lin_tl_model.predict(x_test_t_idf_index)
    s_mean_lin_tl_model, s_var_lin_tl_model = lin_tl_model.predict(x_test_s_idf_index)


    print(t_mean_lin_tl_model.shape)
    r2_tl = r2_score(y_test_t,t_mean_lin_tl_model )
    mse_tl = mean_squared_error(y_test_t,t_mean_lin_tl_model )
    print(f'r2 score for multifidelity linear {r2_score(y_test_t,t_mean_lin_tl_model )}')
    print(f'mse for multifidelity linear {mean_squared_error(y_test_t,t_mean_lin_tl_model )}')

    

    return lin_tl_model

In [32]:
# residue_df = pd.DataFrame()
# n_batch = 10
# df_results = pd.DataFrame(columns=["observable", "fold","n_training","r2_lin_tl","r2_stand_GP",
#                                   "mse_lin_tl","mse_stand_GP","wn_lin_tl","wn_stand_GP","rho"])
# split_i = 0
# lin_tl_emulators=[]
# for selected_observable in observables_choosen:
#     start_time = time.time()
#     Y_s=simulation_df[0][selected_observable].values.reshape(-1,1)
#     Y_t=simulation_df_Au[0][selected_observable].values.reshape(-1,1)
#     Y_v=simulation_df_Au_validation[selected_observable].values.reshape(-1,1)
    
#     x_train_t, x_test_t, y_train_t, y_test_t = XAu, Xv, Y_t, Y_v
#     x_train_s, y_train_s = X, Y_s
    
#     for i in range(0,n_batch):
#         if i in [5]:
#             l=0
#             h=(x_train_t.shape[0]//n_batch)*(i+1)
#         else:
#             continue
#         if i == n_batch-1:
#             h=x_train_t.shape[0]
#         lin_tl_emulators.append(run_train_and_validation(x_train_s, x_train_t[l:h,:],x_test_t, y_train_s, y_train_t[l:h,:], y_test_t, selected_observable))
#     end_time =time.time()
#     print(f'Time it take for {selected_observable} is {end_time-start_time}')
#    time_ar.append(end_time-start_time)
#time_ar= np.array(time_ar)


In [33]:
#import pickle
#with open(data_path('AuAusobol'),"wb") as f:
#    pickle.dump(lin_tl_emulators,f)

In [65]:
import pickle
lin_tl_emulators = pickle.load(open(data_path('AuAusobol'),"rb"))

In [66]:
lin_tl_emulators[0].Y

ObsAr([[ 0.61678436],
       [-0.17583479],
       [ 1.03168586],
       [ 0.13991935],
       [ 0.68908011],
       [ 0.20188636],
       [ 0.18918553],
       [-0.01543866],
       [ 0.17675179],
       [ 0.83655083],
       [ 0.93253248],
       [-0.05606746],
       [ 0.65727484],
       [ 0.03010433],
       [ 0.85921088],
       [-0.014759  ],
       [ 0.78154887],
       [ 0.39349891],
       [-0.78023822],
       [ 0.28886651],
       [-0.53449784],
       [ 0.89784736],
       [-0.25880017],
       [ 0.7121214 ],
       [-0.82866515],
       [ 1.09087361],
       [-0.48436278],
       [ 1.2831256 ],
       [ 0.46098072],
       [-0.08164693],
       [-0.2367711 ],
       [-0.33116213],
       [-0.60718108],
       [ 0.2883079 ],
       [-0.37070488],
       [ 1.300795  ],
       [-0.00612407],
       [ 0.09395173],
       [ 0.31318145],
       [-0.47855666],
       [ 0.93445037],
       [ 0.25538921],
       [-0.99114829],
       [ 0.45956721],
       [ 0.88289025],
       [ 0

### We have the emulators. Now let's do a sensitivity study. (Global Sobel index)

In [67]:
prior_df_Au = prior_df

In [68]:
prior_df_Au['norm']['max'] = 10
prior_df_Au['norm']['min'] = 3



In [69]:
prior_df_Au

,norm,trento_p,sigma_k,nucleon_width,dmin3,tau_R,alpha,eta_over_s_T_kink_in_GeV,eta_over_s_low_T_slope_in_GeV,eta_over_s_high_T_slope_in_GeV,eta_over_s_at_kink,zeta_over_s_max,zeta_over_s_T_peak_in_GeV,zeta_over_s_width_in_GeV,zeta_over_s_lambda_asymm,shear_relax_time_factor,Tswitch
min,3.0,-0.7,0.3,0.5,0.000,0.3,-0.3,0.13,-2.0,-1.0,0.01,0.01,0.12,0.025,-0.8,2.0,0.130
max,10.0,0.7,2.0,1.5,4.913,2.0,0.3,0.30,1.0,2.0,0.20,0.20,0.30,0.150,0.8,8.0,0.165


In [70]:
prior_df_Au.values.T.shape

(17, 2)

In [71]:
from SALib.sample import saltelli
from SALib.analyze import sobol

In [72]:
problem = {
    'groups' : ['$N$[$200$GeV]', 'IC', 'IC', 'IC','IC', 'FS', 'FS', '$(\\eta/s)$','$(\\eta/s)$','$(\\eta/s)$','$(\\eta/s)$',
               '$(\\zeta/s)$','$(\\zeta/s)$','$(\\zeta/s)$','$(\\zeta/s)$','$(\\eta/s)$', '$T_{\\mathrm{sw}}$ [GeV]'],
    'num_vars': 17,
    'names': model_param_dsgn,
    'bounds': prior_df_Au.values.T
}

In [73]:
param_values = saltelli.sample(problem, 16384)


In [74]:
param_values.shape

(229376, 17)

In [ ]:
#Y = np.zeros([param_values.shape[0]])
sns.set_style("whitegrid")
sns.set_context('poster')
Si_array = []
X_train_sb, Y_train_sb = convert_xy_lists_to_arrays([param_values, param_values], [param_values, param_values])
for i, obs in enumerate(observables_choosen):
    print(obs)
    Yt,_ = lin_tl_emulators[i].predict(X_train_sb[param_values.shape[0]:,:])
    Ys,_ = lin_tl_emulators[i].predict(X_train_sb[0:param_values.shape[0],:])
    Ys = Ys *lin_tl_emulators[i].gpy_model.multifidelity.param_array[-1]
    discrp = Yt-Ys
    Y = discrp.flatten()
    Si = sobol.analyze(problem, Y)
    print(Si['S1'])
    print(Si['ST'])
    Si_array.append(Si)
    #sns.set_context('talk')
    fig ,ax = plt.subplots(3,1, figsize=(30,30))
    fig.suptitle(obs)
    axes = Si.plot(ax=ax)
    plt.tight_layout()
    fig.savefig(f'AuAUGrad_{i}_sobolgroup.png', dpi = 50)

dN_dy_pion[0 5]
[ 9.45935975e-01  1.12843625e-02  7.91168308e-09  2.04829552e-02
  2.11693022e-02 -6.34627027e-12]
[9.47037543e-01 1.13329090e-02 8.83343129e-09 2.12896635e-02
 2.13720311e-02 6.81474691e-15]
dN_dy_pion[40 50]
[0.84435804 0.06306864 0.03325901 0.00941388 0.02538639 0.01513422]
[0.85376095 0.06968244 0.03343847 0.00966938 0.02622702 0.01672436]
mean_pT_pion[0 5]
[ 6.10191854e-01  7.12146993e-07  1.12336929e-01  1.00931269e-01
  1.35333091e-01 -1.17091551e-09]
[6.34466745e-01 7.15475917e-07 1.45117606e-01 1.25174800e-01
 1.41738422e-01 1.47130898e-13]
mean_pT_pion[40 50]


In [ ]:
with open(data_path('AuAusobolgroup_index'),"wb") as f:
    pickle.dump(Si_array,f)

In [ ]:
#with open(data_path('AuAusobolgroup_index'),"rb") as f:
#    Si_array_loaded=pickle.load(f)